# This is the first step in reproducing the cluster analysis from Sara and David following Amini et al 2019
https://doi.org/10.1007/s00382-018-4409-7

This program:
1. Reads in Z500 and U250 from ERA-Interim for Dec-Jan-Feb-Mar 1980/81 to 2014/15
* Data are located in: 
`/shared/working/rean/era-interim/daily/data/yyyy/ei.oper.an.pl.regn128cm.yyyymmddhh`
2. Subsets to the PNA region
* Defined as: 150-300E; 20-80N
3. Interpolates the data to a 126x64 Gaussian Grid
4. Writes datasets out to a netcdf files containing both variables 
* Output file is located in: `/project/predictability/kpegion/wxregimes/era-interim/erai.z500_u250_pna_5dyrm_DJF.1980-2015.nc`

In [1]:
import warnings

import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob

In [3]:
def preprocZ(ds):

    # Extract PNA region and 500hPa level
    ds=ds.sel(isobaricInhPa=500,latitude=slice(80,20),longitude=slice(150,300))

    # Create a new xarray dataset for z500
    ds_tmp=xr.DataArray(ds['z'],
                        coords={'lat':ds['latitude'].values,
                                'lon': ds['longitude'].values},
                            dims=['lat','lon'])        
    ds_tmp=ds_tmp.to_dataset(name='z500')
    
    # Interpolate data to coarser grid
    ds_new_grid=xr.open_dataset('tempgaussian.128x64.nc')
    ds_new_grid=ds_new_grid.rename({'temp':'z500'}).sel(lat=slice(80,20),lon=slice(150,300))
    ds=ds_tmp.interp_like(ds_new_grid)
    
    return ds

In [4]:
def preprocU(ds):

    # Extract PNA region and 250hPa level
    ds=ds.sel(isobaricInhPa=250,latitude=slice(80,20),longitude=slice(150,300))

    # Create a new xarray dataset for u250
    ds_tmp=xr.DataArray(ds['u'],
                    coords={'lat':ds['latitude'].values,
                            'lon': ds['longitude'].values},
                            dims=['lat','lon'])        
    ds_tmp=ds_tmp.to_dataset(name='u250')

    # Interpolate data to coarser grid
    ds_new_grid=xr.open_dataset('tempgaussian.128x64.nc')
    ds_new_grid=ds_new_grid.rename({'temp':'u250'}).sel(lat=slice(80,20),lon=slice(150,300))
    ds=ds_tmp.interp_like(ds_new_grid)
    
    return ds

### Define months and years to get

In [5]:
# Months list
mnums=['01','02','03','12']

# Dates
sdate='19800101'
edate='20151231'

# Years List
yrs_list=np.arange(1980,2016)
yrs_list

array([1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990,
       1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001,
       2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015])

### Define file path and names

In [6]:
# Input
path='/shared/working/rean/era-interim/daily/data/'
fname='ei.oper.an.pl.regn128cm.'

# Output
out_path='/project/predictability/kpegion/wxregimes/era-interim/'
ofname='erai.z500_u250_pna_DJF.1980-2015.nc'
outfile=out_path+ofname

### Read in the files for all months and years, and prepocess to subset data for Z500hPa and U250 in the PNA region & interpolate to coarser grid

In [7]:
# Create empty list to append data for each month
ds_Z_months=[]
ds_U_months=[]

# Loop over months, get list of files and read in data for each month
for mnum in mnums:
        
    
    # Get all the filenames for this month for all years
    fnamesZ = [f'{path}{year}/{fname}{year}{mnum}*' for year in yrs_list]       
   
    # Create list of all filenames for this month, and all years
    filesZ=[]
    for files in fnamesZ:
        f2=glob.glob(files)
        for f in f2:
            filesZ.append(f)
    
    filesZ=sorted(filesZ)
    #print(filesZ)
    
    # Read Geopotential Height (Z) data           
    ds_Z=xr.open_mfdataset(filesZ,engine='cfgrib',
                           combine='nested',concat_dim=['time'],
                           backend_kwargs={'indexpath':'',
                                           'filter_by_keys':{'name': 'Geopotential'}},
                           preprocess=preprocZ)
 
    # Read zonal wind (U) data
    ds_U=xr.open_mfdataset(filesZ,engine='cfgrib',
                           combine='nested',concat_dim=['time'],
                           backend_kwargs={'indexpath':'',
                                           'filter_by_keys':{'name': 'U component of wind'}},
                           preprocess=preprocU) 

    
    # Create dates for time assign to time dimension
    dates_all=pd.date_range(start=sdate,end=edate,freq='D')
    print(dates_all)
    djf_dates=dates_all[(dates_all.month==int(mnum))]
    print(djf_dates)
    ds_Z['time']=djf_dates
    ds_U['time']=djf_dates
        
    # Append the latest month to the list
    ds_Z_months.append(ds_Z)
    ds_U_months.append(ds_U)
        
# Combine the months into the init dimension
ds_Z_months = xr.combine_nested(ds_Z_months, concat_dim='time')
ds_U_months = xr.combine_nested(ds_U_months, concat_dim='time')

DatetimeIndex(['1980-01-01', '1980-01-02', '1980-01-03', '1980-01-04',
               '1980-01-05', '1980-01-06', '1980-01-07', '1980-01-08',
               '1980-01-09', '1980-01-10',
               ...
               '2015-12-22', '2015-12-23', '2015-12-24', '2015-12-25',
               '2015-12-26', '2015-12-27', '2015-12-28', '2015-12-29',
               '2015-12-30', '2015-12-31'],
              dtype='datetime64[ns]', length=13149, freq='D')
DatetimeIndex(['1980-01-01', '1980-01-02', '1980-01-03', '1980-01-04',
               '1980-01-05', '1980-01-06', '1980-01-07', '1980-01-08',
               '1980-01-09', '1980-01-10',
               ...
               '2015-01-22', '2015-01-23', '2015-01-24', '2015-01-25',
               '2015-01-26', '2015-01-27', '2015-01-28', '2015-01-29',
               '2015-01-30', '2015-01-31'],
              dtype='datetime64[ns]', length=1116, freq=None)
DatetimeIndex(['1980-01-01', '1980-01-02', '1980-01-03', '1980-01-04',
               '1980-01-0

### Combine z500 and u250 together into single `xr.Dataset`

In [8]:
ds_ZU=xr.merge([ds_Z_months,ds_U_months])

### Write to netcdf

In [11]:
ds_ZU.to_netcdf(outfile)